In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

## Preprocessing

In [10]:
import string
import regex as re
import numpy as np
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


def preprocess(text):
    # Remove integers
    text = re.sub(r'\d+', '', text)

    # remove newlines as \r and \n
    text = re.sub(r'\r', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation marks
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    return text


def encode_text_and_labels(df):
    t = Tokenizer()
    t.fit_on_texts(df['text'])
    # keep only words that appear more than min_df times
    # keep only words that appear more than min_df times
    word_docs = {word: freq for word, freq in t.word_docs.items() if freq > 5}

    # create new tokenizer
    t_filtered = Tokenizer()

    # fit t_filtered
    t_filtered.word_index = {word: i + 1 for i, (word, freq) in enumerate(word_docs.items())}
    t_filtered.word_counts = word_docs

    vocab_size = len(t_filtered.word_index) + 1
    # integer encode the documents
    encoded_docs = t_filtered.texts_to_sequences(df['text'])
    # pad documents to be as long as the longest sequence in the dataset
    max_length = df['text'].apply(lambda x: len(x.split(' '))).max()
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(df['artist'])
    # binary encode
    onehot_encoded = to_categorical(integer_encoded)
    return padded_docs, onehot_encoded, vocab_size, max_length

def load_and_preprocess_data(path, in_colab=False):
    """
    Load the data and preprocess it, expect runtime of 20 seconds.
    :param path: path to the data
    :return: preprocessed data in the form of a pandas dataframe. The first item returned is the data,
    the second is the labels, the third is the vocabulary size, and the fourth is the maximum length of a sequence
    """
    df = pd.read_csv(path)
      
    # remove artist with fewer than 30 songs
    df = df.groupby('artist').filter(lambda x: len(x) > 100)

    df['text'] = df['text'].apply(preprocess)

    # Identify the rows that contain duplicated text in the 'song' column
    no_covers = ~df['song'].duplicated()

    # Filter the DataFrame to include only the rows with unique text
    df = df[no_covers]

    # prepare text data for a recurrent network
    return encode_text_and_labels(df)



In [11]:
padded_docs, artists_onehot_encoded, vocab_size, max_length = load_and_preprocess_data("/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv")

In [12]:
X_train, X_test, y_train, y_test = train_test_split(padded_docs, artists_onehot_encoded, test_size=0.2, random_state=42)

In [13]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [14]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_length, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=max_length, output_dim=embed_dim)

    def call(self, x):
        max_length = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=max_length, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [15]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(artists_onehot_encoded.shape[1], activation="softmax")(x)

transformer = keras.Model(inputs=inputs, outputs=outputs)

In [16]:
transformer.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
transformer.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 950)]             0         
_________________________________________________________________
token_and_position_embedding (None, 950, 32)           491968    
_________________________________________________________________
transformer_block_1 (Transfo (None, 950, 32)           10656     
_________________________________________________________________
global_average_pooling1d_1 ( (None, 32)                0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0   

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5,
                                                    restore_best_weights=True,
                                                    verbose=1)]

history = transformer.fit(
    X_train,
    y_train,
    validation_split=0.1,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
    use_multiprocessing=True
)

Epoch 1/200
352/352 [==============================] - 27s 73ms/step - loss: 5.5880 - accuracy: 0.0044 - val_loss: 5.5824 - val_accuracy: 0.0044
Epoch 2/200
352/352 [==============================] - 25s 72ms/step - loss: 5.4851 - accuracy: 0.0085 - val_loss: 5.4044 - val_accuracy: 0.0076
Epoch 3/200
352/352 [==============================] - 26s 73ms/step - loss: 5.3608 - accuracy: 0.0120 - val_loss: 5.3329 - val_accuracy: 0.0132
Epoch 4/200
352/352 [==============================] - 26s 73ms/step - loss: 5.2967 - accuracy: 0.0137 - val_loss: 5.3101 - val_accuracy: 0.0172
Epoch 5/200
327/352 [==========================>...] - ETA: 1s - loss: 5.2444 - accuracy: 0.0159